In [1]:
import pandas as pd
import numpy as np
import os
import math
import json

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [3]:
defense = pd.read_csv("../match_logs/Big5@22-23@defense.csv")
gca = pd.read_csv("../match_logs/Big5@22-23@gca.csv")
keeper = pd.read_csv("../match_logs/Big5@22-23@keeper.csv")
misc = pd.read_csv("../match_logs/Big5@22-23@misc.csv")
passing = pd.read_csv("../match_logs/Big5@22-23@passing.csv")
passing_types = pd.read_csv("../match_logs/Big5@22-23@passing_types.csv")
possession = pd.read_csv("../match_logs/Big5@22-23@possession.csv")
shooting = pd.read_csv("../match_logs/Big5@22-23@shooting.csv")

In [1]:
pd.read_csv("../players_db/fm23/fm23db_processed.csv").sort_values(by="tpr",ascending=False)[["Club","Name","Best_Pos","tpr"]].head(25)

NameError: name 'pd' is not defined

In [5]:
team_ratings = pd.read_csv("../players_db/fm23/team_ratings.csv")

In [6]:
team_ratings_cols = team_ratings.columns

In [7]:
team_ratings.sort_values("tpr", ascending=False).head(n=10)

,Club,GK,Based,Division,Club_id,League_id,tpr,Anticipation,Teamwork,Corners,...,Heading,Decisions,Tackling,Marking,Leadership,Composure,Determination,Passing,First_Touch,fbref_name
3,Manchester City,72,England (Premier Division),English Premier Division,6827,354,92,88,72,78,...,71,88,88,85,74,88,83,87,85,Manchester City
0,FC Bayern,75,Germany (Bundesliga),Bundesliga,3704,185,89,86,86,72,...,71,80,82,80,87,84,86,75,71,Bayern Munich
51,Paris Saint-Germain,63,France (Ligue 1 Uber Eats),Ligue 1 Uber Eats,7994,773,89,81,76,79,...,72,86,82,83,81,84,85,87,87,Paris S-G
26,AC Milan,65,Italy (Serie A),Italian Serie A,7093,710,89,80,88,69,...,80,71,78,85,92,74,92,71,80,Milan
20,Internazionale,66,Italy (Serie A),Italian Serie A,5215,710,89,81,92,80,...,92,72,90,92,78,65,90,80,75,Inter
2,Liverpool,72,England (Premier Division),English Premier Division,6518,354,86,81,88,78,...,82,82,86,89,90,81,85,83,87,Liverpool
6,Real Madrid,68,Spain (First Division),Spanish First Division,8716,1215,86,83,80,83,...,74,90,82,87,82,84,83,92,92,Real Madrid
9,Juventus,68,Italy (Serie A),Italian Serie A,5523,710,86,83,80,78,...,72,65,80,76,85,71,85,85,75,Juventus
70,Lazio,61,Italy (Serie A),Italian Serie A,6346,710,86,83,80,75,...,80,74,78,83,81,64,92,75,75,Lazio
1,FC Barcelona,74,Spain (First Division),Spanish First Division,1435,1215,86,92,86,71,...,80,90,80,89,82,85,78,80,82,Barcelona


In [8]:
tpr_cols = []
except_cols = ["Club","Based","Division","Club_id","League_id","fbref_name"]
for col in team_ratings_cols:
    if col not in except_cols:
        for xy in ["x","y"]:
            xy_col = f"{col}_{xy}"
            tpr_cols.append(xy_col)

<br><br><br><br><br><br><br><br>
<h1 style="color:orange;background:blue;">@.  Logs to Regression</h1>

In [9]:
CURRENT_DATAFRAME_NAME = "passing"
attribute = "Att"

In [10]:
CURRENT_DATAFRAME = locals()[CURRENT_DATAFRAME_NAME]
CURRENT_DATAFRAME["Unnamed: 0"] = "itsanobject"

In [11]:
# CURRENT_DATAFRAME["Unique_Matchday_id"] = ((CURRENT_DATAFRAME.Club).apply(hash)+(CURRENT_DATAFRAME.Opponent).apply(hash)).apply(str)
# unique_indexes = CURRENT_DATAFRAME["Unique_Matchday_id"].drop_duplicates().index
# CURRENT_DATAFRAME = CURRENT_DATAFRAME.loc[unique_indexes]

In [12]:
DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.columns)
NUMERIC_DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.select_dtypes(exclude="object").columns)

In [13]:
CURRENT_DATAFRAME[NUMERIC_DATAFRAME_COLUMNS].describe().loc[['count','min', 'mean', 'max']].T[["mean"]].apply(round)

,mean
GF,1.0
GA,1.0
Cmp,385.0
Att,487.0
Cmp%,78.0
TotDist,6880.0
PrgDist,2508.0
Cmp.1,171.0
Att.1,196.0
Cmp%.1,86.0


In [14]:
merged_df_1 = pd.merge(CURRENT_DATAFRAME, team_ratings, on="Club", how="inner")  
merged_df_2 = pd.merge(merged_df_1, team_ratings, left_on="Opponent", right_on="fbref_name", how="inner") 
merged_df = merged_df_2.copy()

In [15]:
merged_df["Versus_Title"] = merged_df.Club_x + " plays vs " + \
                             merged_df.Club_y + " as " + merged_df.Venue + " team"
merged_df["tpr_diff"] = merged_df.tpr_x - merged_df.tpr_y
merged_df["tpr_diff_abs"] = abs(merged_df.tpr_x - merged_df.tpr_y)
merged_df["goal_diff"] = merged_df.GF - merged_df.GA
merged_df["goal_diff_abs"] = abs(merged_df.GF - merged_df.GA)

In [16]:
# merged_df = merged_df.query("tpr_diff < 3")
merged_df = merged_df.query("goal_diff > 0 and tpr_diff > 1")
merged_df.describe()

,GF,GA,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,...,Marking_y,Leadership_y,Composure_y,Determination_y,Passing_y,First_Touch_y,tpr_diff,tpr_diff_abs,goal_diff,goal_diff_abs
count,821.000000,821.000000,821.000000,821.000000,821.000000,821.00000,821.000000,821.000000,821.000000,821.000000,...,821.000000,821.000000,821.000000,821.000000,821.000000,821.000000,821.000000,821.000000,821.000000,821.000000
mean,2.493301,0.556638,444.677223,542.154689,80.765773,7752.09866,2720.084044,204.708892,229.204629,88.244336,...,72.714982,69.538368,63.488429,69.585871,65.505481,63.253350,9.789281,9.789281,1.936663,1.936663
std,1.277962,0.719600,132.583004,127.050227,6.502196,2059.42042,485.415833,73.581541,74.001101,4.730890,...,6.519886,7.507073,5.472452,8.025964,4.932375,5.382141,6.237822,6.237822,1.181517,1.181517
min,1.000000,0.000000,124.000000,234.000000,50.900000,2256.00000,1053.000000,52.000000,74.000000,69.100000,...,61.000000,50.000000,53.000000,50.000000,58.000000,54.000000,3.000000,3.000000,1.000000,1.000000
25%,2.000000,0.000000,341.000000,446.000000,76.700000,6204.00000,2380.000000,149.000000,174.000000,85.500000,...,67.000000,65.000000,59.000000,65.000000,62.000000,59.000000,6.000000,6.000000,1.000000,1.000000
50%,2.000000,0.000000,434.000000,532.000000,81.600000,7645.00000,2725.000000,195.000000,220.000000,89.200000,...,72.000000,68.000000,64.000000,71.000000,66.000000,64.000000,9.000000,9.000000,2.000000,2.000000
75%,3.000000,1.000000,534.000000,630.000000,85.700000,9152.00000,3038.000000,252.000000,275.000000,91.700000,...,76.000000,74.000000,67.000000,74.000000,67.000000,67.000000,12.000000,12.000000,2.000000,2.000000
max,9.000000,4.000000,817.000000,917.000000,93.300000,13597.00000,4221.000000,446.000000,473.000000,97.400000,...,92.000000,90.000000,92.000000,92.000000,85.000000,87.000000,30.000000,30.000000,9.000000,9.000000


In [17]:
MERGED_NUMERIC_DATAFRAME_COLUMNS = list(merged_df.select_dtypes(exclude="object").columns)

In [18]:
reg_df = merged_df[['Club_x','Club_y',"Versus_Title","Date"] + MERGED_NUMERIC_DATAFRAME_COLUMNS] 
reg_df.iloc[0]

Club_x                                             Manchester City
Club_y                                             West Ham United
Versus_Title     Manchester City plays vs West Ham United as Aw...
Date                                                    2022-08-07
GF                                                             2.0
                                       ...                        
First_Touch_y                                                   74
tpr_diff                                                        15
tpr_diff_abs                                                    15
goal_diff                                                      2.0
goal_diff_abs                                                  2.0
Name: 0, Length: 112, dtype: object

<br><br><br><br><br><br><br><br>
<h2 style="color:white;background:red;">  Regression to get Feature Importance</h2>

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
opponent_cols = [col for col in tpr_cols if "_x" in col]
opponent_cols.append("tpr_x")

In [21]:
formula_df = pd.DataFrame(index=opponent_cols)#pd.DataFrame(index=tpr_cols)
np.random.seed(0)
df = reg_df.dropna(axis=0)
NUMERIC_DATAFRAME_COLUMNS = [attribute] #only attribute

for attr in NUMERIC_DATAFRAME_COLUMNS:
    X = df[opponent_cols] #df[tpr_cols]
    Y = df[[attr]]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    coef_column_name = f"{attr}_coef"
    intercept_column_name = f"{attr}_intercept"
    formula_df[coef_column_name] = model.coef_.flatten()
    formula_df[intercept_column_name] = model.intercept_[0]

    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    print(f'{attr} --- Mean Squared Error: {mse}')
    print(f'{attr} --- R^2 Score: {r2}')
    # print("\n")

Att --- Mean Squared Error: 10341.38486803669
Att --- R^2 Score: 0.2907914678980801


In [22]:
fdf_attribute = f"{attribute}_coef"
absolute_fdf_attribute = f"abs_{attribute}_coef"

formula_df[absolute_fdf_attribute] = formula_df[fdf_attribute].apply(abs)
sorted_formula_df = formula_df.query(f"{absolute_fdf_attribute} > 0.1").sort_values(by=absolute_fdf_attribute, ascending=False)
sorted_formula_df

,Att_coef,Att_intercept,abs_Att_coef
Technique_x,11.316717,166.347436,11.316717
Anticipation_x,6.028317,166.347436,6.028317
Free_Kick_Taking_x,-5.584081,166.347436,5.584081
Dribbling_x,-5.093579,166.347436,5.093579
Passing_x,-3.871694,166.347436,3.871694
Corners_x,3.787897,166.347436,3.787897
Positioning_x,-3.740391,166.347436,3.740391
Agility_x,3.040096,166.347436,3.040096
Strength_x,-2.900159,166.347436,2.900159
Concentration_x,-2.876391,166.347436,2.876391


In [23]:
sorted_formula_df.loc[:, fdf_attribute]

Technique_x           11.316717
Anticipation_x         6.028317
Free_Kick_Taking_x    -5.584081
Dribbling_x           -5.093579
Passing_x             -3.871694
Corners_x              3.787897
Positioning_x         -3.740391
Agility_x              3.040096
Strength_x            -2.900159
Concentration_x       -2.876391
Crossing_x            -2.636202
Pace_x                 2.582556
Determination_x       -2.232790
Decisions_x            2.160027
GK_x                   2.072710
Teamwork_x            -2.009854
Tackling_x             1.807047
Long_Throws_x          1.630380
Flair_x               -1.355027
tpr_x                  1.326109
tpr_x                  1.326109
Stamina_x             -1.166083
First_Touch_x          1.076169
Work_Rate_x           -1.016397
Off_the_Ball_x         0.999007
Long_Shots_x          -0.938233
Marking_x             -0.907802
Composure_x            0.848963
Acceleration_x         0.843046
Aggression_x          -0.827644
Penalty_Taking_x       0.793141
Finishin

In [24]:
formula_df.to_excel(f"{CURRENT_DATAFRAME_NAME.upper()}_COEF.xlsx")
# with pd.ExcelWriter(f"{CURRENT_DATAFRAME_NAME.upper()}_COEF.xlsx") as writer:
#     formula_df[formula_df.index.str.contains("_x")].to_excel(writer, index=True, sheet_name="x")
#     formula_df[formula_df.index.str.contains("_y")].to_excel(writer, index=True, sheet_name="y")

<br><br><br><br><br><br><br><br>
<h2 style="color:white;background:green;">  RESEARCHING MATHEMATICS</h2>

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Load your data
data = merged_df

# Define features and target
features = tpr_cols
target = attribute

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.24, random_state=42)

# Initialize and train the model
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 8106.587637565757


In [26]:
reg_df.query("Versus_Title == 'Manchester City plays vs Aston Villa as Home team'").iloc[0].name

191

In [27]:
reg_df.loc[191]

Club_x                                             Manchester City
Club_y                                                 Aston Villa
Versus_Title     Manchester City plays vs Aston Villa as Home team
Date                                                    2023-02-12
GF                                                             3.0
                                       ...                        
First_Touch_y                                                   67
tpr_diff                                                        18
tpr_diff_abs                                                    18
goal_diff                                                      2.0
goal_diff_abs                                                  2.0
Name: 191, Length: 112, dtype: object

In [28]:
prediction_gbr = model.predict([reg_df.iloc[191][features]])[0]

print(f'Gradient Boosting Regressor Prediction: {prediction_gbr}')

Gradient Boosting Regressor Prediction: 589.1095362546362


C:\Users\okany\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


<br><br><br><br>

<br><br><br><br><br><br><br><br>
<h2 style="color:#33FF49;background:blue;">  Regression for One Atrribute</h2>

In [29]:
# tpr_x_dict = {
#     "tpr_x": None,
#     "Acceleration_x": None,
#     "Technique_x": None,
#     "Finishing_x": None,
#     "Composure_x": None,
#     "Agility_x": None,
#     "Long_Shots_x": None,
#     "Off_the_Ball_x": None,
#     "Teamwork_x": None,
#     "Work_Rate_x": None,
#     "First_Touch_x": None
# }
tpr_x_dict = {"tpr_x": None}

In [30]:
# tpr_y_dict = {
#     "tpr_y": None,
#     "Acceleration_y": None,
#     "Off_the_Ball_y": None,
#     "Agility_y": None,
#     "Work_Rate_y": None,
#     "Decisions_y": None,
#     "Teamwork_y": None,
#     "Balance_y": None,
#     "Positioning_y": None,
#     "Marking_y": None
# }
tpr_y_dict = {"tpr_y": None}
# for col in opponent_cols:
#     tpr_y_dict[col] = None

In [31]:
# https://fbref.com/en/matches/5965a1a5/Bayern-Munich-Bochum-February-11-2023-Bundesliga
# row_index= 3221 # for Bayern
# row_index2222= 3758 # for Bochum

row_index222 = 1# 687 # for Liverpool
row_index = 190#356 # for Arsenal

x_cols = tpr_x_dict.keys()
y_cols = tpr_y_dict.keys()

tpr_x_custom_data = reg_df[x_cols].iloc[row_index].to_dict()
tpr_y_custom_data = reg_df[y_cols].iloc[row_index].to_dict()

custom_data = {**tpr_x_custom_data, **tpr_y_custom_data}
custom_cols = custom_data.keys()

In [32]:
custom_cols

dict_keys(['tpr_x', 'tpr_y'])

In [33]:
reg_df.iloc[row_index]

Club_x                                        Lazio
Club_y                                       Spezia
Versus_Title     Lazio plays vs Spezia as Away team
Date                                     2023-04-14
GF                                              3.0
                                ...                
First_Touch_y                                    56
tpr_diff                                         18
tpr_diff_abs                                     18
goal_diff                                       3.0
goal_diff_abs                                   3.0
Name: 803, Length: 112, dtype: object

In [34]:
reg_df[x_cols].iloc[row_index]

tpr_x    86
Name: 803, dtype: int64

In [35]:
# q1 = reg_df.query("Versus_Title == 'Manchester City plays vs Liverpool as Home team'")
# q2 = reg_df.query("Versus_Title == 'Liverpool plays vs Manchester City as Home team'")
# queries = ["", q1, q2]
# query_no = 2
# print(queries[query_no].iloc[0].Date)
# queries[query_no].iloc[0].name
# queries[query_no].iloc[0].name

In [36]:
special_formula_df = pd.DataFrame(index=custom_cols)

df_to_custom = df.copy()
X = df_to_custom[custom_cols]
Y = df_to_custom[[attribute]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

coef_column_name = f"{attribute}_coef"
intercept_column_name = f"{attribute}_intercept"
special_formula_df[coef_column_name] = model.coef_.flatten()
special_formula_df[intercept_column_name] = model.intercept_[0]

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

In [37]:
special_formula_df.sort_values(by=fdf_attribute, ascending=False)

,Att_coef,Att_intercept
tpr_x,9.932326,145.213431
tpr_y,-5.482631,145.213431


In [38]:
custom_tpr_df = pd.DataFrame(custom_data, index=["tpr_value"]).T

In [39]:
custom_tpr_df[f"{attribute}_coef"] = special_formula_df[f"{attribute}_coef"][custom_cols].tolist()
custom_tpr_df[f"{attribute}_intercept"] = special_formula_df[f"{attribute}_intercept"][custom_cols].tolist()
custom_tpr_df["coef_result"] = custom_tpr_df[f"{attribute}_coef"] * custom_tpr_df["tpr_value"]
custom_tpr_df["formula_result"] = custom_tpr_df["coef_result"].sum() + custom_tpr_df[f"{attribute}_intercept"].iloc[0]
custom_tpr_df["formula_result"] = custom_tpr_df["formula_result"].iloc[0]#round(custom_tpr_df["formula_result"].iloc[0])
custom_tpr_df["formula_result_per_minute"] = custom_tpr_df["formula_result"].iloc[0] / 90
round(custom_tpr_df["formula_result"].iloc[0])

627

In [40]:
prob_of_shot_in_a_minute = custom_tpr_df["formula_result_per_minute"].iloc[0]

def ProbAction(prob_of_shot_in_a_minute):
    r = np.random.choice([1, 0], p=[prob_of_shot_in_a_minute, 1-prob_of_shot_in_a_minute])
    return r

In [42]:
counter = 0

for minute in range(1, 91):
    result = ProbAction(prob_of_shot_in_a_minute)
    counter+=result

counter

ValueError: probabilities are not non-negative

In [ ]:
custom_tpr_df.loc[:, "tpr_value"]

In [ ]:
custom_tpr_df.index

In [ ]:
json.loads('["x","y","z"]')

<br><br><br><br><br><br><br><br>
<h2 style="color:blue;background:aqua;">  Error Rates</h2>

In [ ]:
def Action_Volume_Predict(attribute, formula_df, input_df):
    coefficients = sum(special_formula_df.loc[:, f"{attribute}_coef"] * input_df[custom_cols])
    result = (coefficients + special_formula_df.loc[:, f"{attribute}_intercept"])
    return result.iloc[0]

In [ ]:
error_df = reg_df.dropna()[["Club_x","Club_y","tpr_x","tpr_y","Att"]]
error_df.describe()

In [ ]:
error_df["Att_min"] = error_df["Att"] / 90
error_df["Predicted"] = error_df.apply(lambda row: Action_Volume_Predict(attribute, special_formula_df, row[custom_cols]), axis=1)
error_df["Predicted_min"] = error_df["Predicted"] / 90
error_df["Error"] = error_df["Att"] - error_df["Predicted"]
error_df["Error_min"] = error_df["Att_min"] - error_df["Predicted_min"]

In [ ]:
error_df.mean(numeric_only=True)

In [ ]:
mancity_raw = merged_df_2.dropna()
mancity_raw["tpr_diff"] = mancity_raw.tpr_x - mancity_raw.tpr_y
mancity_raw["tpr_diff_abs"] = abs(mancity_raw.tpr_x - mancity_raw.tpr_y)
mancity = mancity_raw.query("tpr_diff_abs > 5")[["Club_x","Club_y","tpr_x","tpr_y","Att"]]

for n in range(30):
    mancity = mancity_raw.query(f"tpr_diff_abs > {n}")[["Club_x","Club_y","tpr_x","tpr_y","Att"]]
    
    data = mancity.Att
    mancity_att = pd.Series(data)
    mean_val = mancity_att.mean()
    distances = mancity_att - mean_val
    percentage_distances = (distances / mean_val) * 100
    np.random.seed(42)
    randomness = np.random.uniform(-0.1, 0.1, size=percentage_distances.shape) * percentage_distances
    adjusted_values = mancity_att + randomness
    
    df_comparison = pd.DataFrame({
        'Original': mancity_att,
        'Percentages_Distance': percentage_distances,
        'Randomness': randomness,
        'Adjusted': adjusted_values
    })
    
    median = df_comparison.Percentages_Distance.median()
    print(n, median)